In [7]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoModelForCausalLM
from transformers import Qwen3MoeConfig, Qwen3MoeForCausalLM
from experiments_transformermodels import *
import lm_eval
from lm_eval.models.huggingface import HFLM

In [2]:
run_experiment_qwen3_moe(n_samples=100)

Streaming HuggingFaceFW/fineweb-edu (sample-10BT) (samples: 100)...


Resolving data files:   0%|          | 0/2410 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2410 [00:00<?, ?it/s]

Model Parameters: 20,723,488


/home/dylan/projects/moe-experiments/experiments_transformermodels.py:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 151643, 'pad_token_id': 151643}.
/home/dylan/.local/lib/python3.10/site-packages/datasets/formatting/torch_formatter.py:222: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`use_cache=True` is incompatible with gradie

Step,Training Loss
10,15.202200
20,14.633300
30,14.106000
40,14.682000
50,13.223400
60,12.921900
70,13.662900
80,12.509500
90,12.400800
100,13.295800


'(ReadTimeoutError("HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 60efc4f7-56cc-4126-8e97-63fd8af44ced)')' thrown while requesting GET https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu/resolve/87f09149ef4734204d70ed1d046ddc9ca3f2b8f9/data/CC-MAIN-2013-20/train-00000-of-00014.parquet
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f566b453-9e42-40a7-a860-e7888b837111)')' thrown while requesting GET https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu/resolve/87f09149ef4734204d70ed1d046ddc9ca3f2b8f9/data/CC-MAIN-2013-20/train-00000-of-00014.parquet
Retrying in 1s [Retry 1/5].


In [8]:
# Load trained model
model = AutoModelForCausalLM.from_pretrained(
    "./qwen3moe-test",
    dtype="auto",
    device_map="auto",
    trust_remote_code=False
)

eval_model = HFLM(
    pretrained=model, 
    device=device,
    batch_size=8
)

# Choose tasks (Hellaswag for commonsense, ARC-Easy for reasoning)
tasks = ["hellaswag", "arc_easy"]

# Run evaluation
results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=tasks,
    num_fewshot=0,  # Zero-shot evaluation
    limit=1000      # Limit samples for quick testing
)

# Print results
for task, metrics in results["results"].items():
    print(f"{task}: {metrics}")

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
The tokenizer you are loading from './qwen3moe-test' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
Overwriting default num_fewshot of arc_easy from None to 0
Overwriting default num_fewshot of hellaswag from None to 0
Running loglikelihood requests: 100%|██████| 7997/7997 [00:41<00:00, 194.89it/s]


arc_easy: {'alias': 'arc_easy', 'acc,none': 0.256, 'acc_stderr,none': 0.013807775152234187, 'acc_norm,none': 0.255, 'acc_norm_stderr,none': 0.013790038620872828}
hellaswag: {'alias': 'hellaswag', 'acc,none': 0.262, 'acc_stderr,none': 0.013912208651021352, 'acc_norm,none': 0.267, 'acc_norm_stderr,none': 0.013996674851796249}
